In [ ]:
import sys
sys.path.append('/Users/busalels/Desktop/KAUST/spring2024/research')

In [ ]:
from scipy import integrate
import sympy as sp
import numpy as np
from numpy import linalg as la 
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
font = {'size'   : 15}
matplotlib.rc('font', **font)
from New_Homogenized_system_coefficients1_pwc import C_values, Homogenized_system_coef
fft = np.fft.fft
ifft = np.fft.ifft
from tqdm import tqdm 

In [ ]:
delta = 1
gamma = 1.4 
kappa =1
p_0 = 0.3
P1 = gamma*kappa*p_0**(gamma-1)
P11= (gamma-1)*gamma*kappa*p_0**(gamma-2)

L = 1600 
tmax = 900
num_plots = 450

cross_section = 'piecewise'
if cross_section == 'pwc_small':
    a = lambda y: np.piecewise(y, [y- np.floor(y) < 0.5, y- np.floor(y) >= 0.5], [1/2, 3/4])
    dady = lambda y: np.zeros_like(y)
    m = 6000
    Amp = 1/40
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L
    rho0 = 0.3 + Amp*np.exp(-1*(x/10)**2)
    
elif cross_section == 'variable':
    a = lambda y: (3/5) + (2/5) *np.sin(2.*np.pi*y)
    dady = lambda y: (2/5)*2*np.pi*np.cos(2.*np.pi*y)
    m = 6000
    Amp = 1/12 
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L
    rho0 = p_0 + Amp * np.exp(-(x/5)**2) 

elif cross_section == 'piecewise':
    a = lambda y: np.piecewise(y, [y- np.floor(y) < 0.5, y- np.floor(y) >= 0.5], [1/4, 3/4])
    dady = lambda y: np.zeros_like(y)
    m = 6000
    Amp = 1/20
    x = np.arange(-m/2,m/2)*(L/m)
    xi = np.fft.fftfreq(m)*m*2*np.pi/L
    rho0 = 0.3 + Amp*np.exp(-1*(x/8)**2)

In [ ]:
C1, C2, C3, C4, C5, C6, C7, C8, C9,C10,C11,C12,C13,C14,C15,C16,avga,ainvavg = C_values(a=a,dady=dady,delta=delta)

In [ ]:
r1, r2, r3, r4, r5,r5b, r6,r7,r8, beta1, beta2, beta3, beta4, beta5, beta6, beta7,beta8, beta9, beta10, beta11,beta11b = Homogenized_system_coef(C1, C2, C3, C4, C5, C6, C7, C8, C9,C10,C11,C12,C13,C14,C15,C16,avga,ainvavg, p_0, P1, P11, delta)

In [ ]:
q0 = np.zeros_like(rho0)
drhodx = np.zeros_like(rho0)
dqdx = np.zeros_like(rho0)
drhodxt = np.real(ifft(1j*xi*fft(rho0)))
dqdxt = np.real(ifft(1j*xi*fft(q0)))
v0 = np.zeros(2*m)
v0[:m] = rho0
v0[m:] = q0

In [ ]:
def compute_all_transforms(v, xi, m):
    rho = v[:m]
    q = v[m:]
    
    # Fourier Transforms
    rho_hat = fft(rho)
    q_hat = fft(q)
    
    # Spatial Derivatives via Fourier Space
    drhodx = np.real(ifft(1j * xi * rho_hat))
    drho2dx2 = np.real(ifft((1j * xi)**2 * rho_hat))
    drho3dx3 = np.real(ifft((1j * xi)**3 * rho_hat))
    dqdx = np.real(ifft(1j * xi * q_hat))
    dq2dx2 = np.real(ifft((1j * xi)**2 * q_hat))
    
    # Inversion and operations
    inv1 = 1 / (1 - r5b * (1j * xi)**2)
    dqdx1 = np.real(ifft(inv1 * fft(dqdx)))
    dqdxx = np.real(ifft(inv1 * fft(dq2dx2)))
    q_drhodx = np.real(ifft(inv1 * fft(q * drhodx)))
    q2_dqdx = np.real(ifft(inv1 * fft(dqdx * q**2)))
    q_rho_drhodx = np.real(ifft(inv1 * fft(q * rho * drhodx)))
    dqdx_drhodx = np.real(ifft(inv1 * fft(dqdx * drhodx)))
    q_drhodxx = np.real(ifft(inv1 * fft(q * drho2dx2)))
    
    inv2 = 1 / (1 - beta11b * (1j * xi)**2)
    drhodx1 = np.real(ifft(inv2 * fft(drhodx)))
    q_dqdx = np.real(ifft(inv2 * fft(q * dqdx)))
    rho_drhodx = np.real(ifft(inv2 * fft(rho * drhodx)))
    drhodxx = np.real(ifft(inv2 * fft(drho2dx2)))
    q_rho_dqdx = np.real(ifft(inv2 * fft(q * rho * dqdx)))
    dqdx2 = np.real(ifft(inv2 * fft(dqdx**2)))
    q_dqdxx = np.real(ifft(inv2 * fft(q * dq2dx2)))
    q2_drhodx = np.real(ifft(inv2 * fft(drhodx * q**2)))
    drhodx2 = np.real(ifft(inv2 * fft(drhodx**2)))
    rho_drhodxx = np.real(ifft(inv2 * fft(rho * drho2dx2)))

    # Return all the computed transforms
    return {
        'drhodx': drhodx, 'drho2dx2': drho2dx2, 'drho3dx3': drho3dx3, 
        'dqdx': dqdx, 'dq2dx2': dq2dx2, 'dqdx1': dqdx1, 'dqdxx': dqdxx,
        'q_drhodx': q_drhodx, 'q2_dqdx': q2_dqdx, 'q_rho_drhodx': q_rho_drhodx,
        'dqdx_drhodx': dqdx_drhodx, 'q_drhodxx': q_drhodxx, 'drhodx1': drhodx1,
        'q_dqdx': q_dqdx, 'rho_drhodx': rho_drhodx, 'drhodxx': drhodxx,
        'q_rho_dqdx': q_rho_dqdx, 'dqdx2': dqdx2, 'q_dqdxx': q_dqdxx,
        'q2_drhodx': q2_drhodx, 'drhodx2': drhodx2, 'rho_drhodxx': rho_drhodxx
    }


In [ ]:
def rhs_linear(v, xi, m):
    d = np.zeros_like(v)
    
    # Compute all necessary transforms once
    transforms = compute_all_transforms(v, xi, m)
    
    d[:m] = r1 * transforms['dqdx1'] + r2 * transforms['dqdxx']
    d[m:] = beta1 * transforms['drhodx1'] - beta3 * p_0* transforms['drhodx1'] + beta4 * transforms['drhodxx']\
            - beta10 * p_0* transforms['drhodxx']
    
    return d


In [ ]:
def rhs_nonlinear(v, xi, m):
    d = np.zeros_like(v)
    
    # Compute all necessary transforms once
    transforms = compute_all_transforms(v, xi, m)
    
    d[:m] = (r3 * transforms['q_drhodx'] + r4 * transforms['q2_dqdx'] + 
             r6 * transforms['q_rho_drhodx'] - r6 * p_0 * transforms['q_drhodx'] + 
             r7 * transforms['dqdx_drhodx'] + r8 * transforms['q_drhodxx'])
    
    d[m:] = (beta2 * transforms['q_dqdx'] + beta3 * transforms['rho_drhodx'] + 
             beta5 * transforms['q_rho_dqdx'] - beta5 * p_0 * transforms['q_dqdx'] + 
             beta6 * transforms['dqdx2'] + beta7 * transforms['q_dqdxx'] + 
             beta8 * transforms['q2_drhodx'] + beta9 * transforms['drhodx2'] + 
             beta10 * transforms['rho_drhodxx'])
    
    return d


In [ ]:
# Compute the vector for the implicit (linear) part
l = np.zeros(2*m)
l[:m]= 1j*r1*xi + r5*(1j*xi)**3
l[m:] = beta1*1j*xi + beta4 *1j*xi -beta3* p_0*1j*xi -beta10* p_0 *(1j*xi)**2
l = np.hstack((l[:m],l[m:]))

In [ ]:
A    = np.array([[0,0,0],[5/6.,0,0],[11/24,11/24,0]])
Ahat = np.array([[2./11,0,0],[205/462.,2./11,0],[2033/4620,21/110,2/11]])
b    = np.array([24/55.,1./5,4./11])
bhat = b

In [ ]:
def solve_isentropic_gas_equations_ImEx(x,L,m,tmax,dt,xi,num_plots,rhs_linear, rhs_nonlinear,l,v0):
    vv = np.zeros(np.size(v0))
    vv = v0.copy()
    nplt = np.floor((tmax/num_plots)/dt)
    nmax = int(round(tmax/dt))
    frames = [np.reshape(vv.copy(),(2,m))]
    tt = [0]
    progress_bar = tqdm(total=nmax)
    Lfactor = 1/(1-Ahat[0,0]*dt*l)
    D = l
    
    for n in range(1, nmax + 1):
        # Stiff (linear) part treated implicitly
        y1_rhs = (1.0 - dt*A[1, 0]*l)*fft(vv) + dt*A[1, 0]*fft(rhs_linear(vv,xi,m))
        y1 = ifft(y1_rhs)
        Ly1 = rhs_linear(y1,xi,m)
        # Non-stiff (nonlinear) part treated explicitly
        fy1 = rhs_nonlinear(vv,xi,m)
        y2rhs = vv + dt*(A[1,0]*fy1 +Ahat[1,0]*Ly1)
        y2_rhs = (1.0 - dt*A[2,1]*l)*fft(y2rhs) + dt*A[2,1]*fft(rhs_linear(y2rhs,xi,m))
        y2 = ifft(y2_rhs)
        Ly2 = rhs_linear(y2,xi,m)
        fy2 = rhs_nonlinear(y2,xi,m)
        y3rhs = vv + dt*(A[2,0]*fy1 + A[2,1]*fy2 + Ahat[2,0]*Ly1 + Ahat[2,1]*Ly2)
        y3_rhs = (1.0 - dt*b[2]*l)*fft(y3rhs) + dt*b[2]*fft(rhs_linear(y3rhs,xi,m))
        y3 = ifft(y3_rhs)
        Ly3 = rhs_linear(y3,xi,m)
        fy3 = rhs_nonlinear(y3,xi,m)
        v_new = vv + dt*np.real(b[0]*(fy1 + Ly1) +b[1]*(fy2 + Ly2) + b[2]*(fy3 + Ly3))
        vv = v_new.copy()
        vf = np.reshape(vv,(2,m))
        t = n*dt
        # Plotting
        if np.mod(n,nplt) == 0:
            #print(n)
            frames.append(vf.copy())
            tt.append(t)
            progress_bar.update(nplt)
    progress_bar.close()        
    return frames, x, tt, xi



def plot_solution(frames, x, tt, xi):
    fig = plt.figure(figsize=(12,10))
    axes = fig.add_subplot(211)
    axes2 = fig.add_subplot(212)
    line, = axes.plot(x,frames[0][0,:],lw=3)
    line2, = axes2.plot(x,frames[0][1,:],lw=3)
    xi_max = np.max(np.abs(xi))
    #axes.set_xlabel(r'$x$',fontsize=30)
    axes2.set_xlabel(r'$x$',fontsize=30)
    axes.set_ylabel(r'$\rho$',fontsize=30)
    axes2.set_ylabel(r'$q$',fontsize=30)
    #plt.tight_layout()
    plt.close()


    def plot_frame(i):
        line.set_data(x,frames[i][0,:])
        line2.set_data(x,frames[i][1,:])
        axes.set_title('t= %.2e' % tt[i])
        axes.set_xlim((x[0],x[-1]))
        #axes.set_ylim((0.28,0.4))
        axes2.set_xlim((x[0],x[-1]))
        #axes2.set_ylim((-0.15,0.15))
       
        

    anim = matplotlib.animation.FuncAnimation(fig, plot_frame,
                                              frames=len(frames), interval=100,
                                              repeat=False)
    return HTML(anim.to_jshtml())


In [ ]:
dt = 9e-05
print(dt)
frames, x, tt, xi=solve_isentropic_gas_equations_ImEx(x,L,m,tmax,dt,xi,num_plots,rhs_linear, rhs_nonlinear,l,v0)

In [ ]:
plot_solution(frames, x, tt, xi)